In [52]:
from docplex.mp.model import Model  #导出库，只用这一个就够了
import matplotlib.pyplot as plt#选取了用户47的365天的数据
import numpy as np
import pandas as pd
import csv
import cplex
from cplex.exceptions import CplexError
import time
import math
import openpyxl
from openpyxl import Workbook
import xlrd
import torch
from torch import nn
#from matplotlib.animation import FuncAnimation
#import matplotlib.animation as animation
import os
from collections import defaultdict
from random import uniform
from math import sqrt
import matplotlib.pyplot as plt

In [53]:
#读取指定用户的数据：返回GG_array,Total_Load_array,pv_size，此处乘以2，即为功率值
def Read_year(year,user):
    #read file, for example
    file_path = 'C:/Users/chenxihui/Desktop/code/project/'+str(year)+'.csv'
    file = open(file_path)
    data = []
    for i in file.readlines():
        data.append(i)
    file.close()
    new_data = []

    for row in data:
        tmp = row.strip('\n')  #去掉每行最后的回车符
        tmp = tmp.split(',')   #根据','来分割字符串，使之成为含有一个个数据的列表
        new_data.append(tmp)   #new_data的每一行数据就是一个列表
    select_data = []
    #position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    #这是在cluster里分类产生的结果，是人数最多的类，以此为对象
    users=['2','13','14','20','33','35','38','39','56','69','73','74','75'
          ,'82','87','88','101','104','106','109','110','119','124','130'
          ,'137','141','144','152','153','157','161','169','176','184','188'
          ,'189','193','201','202','204','206','207','210','211','212','214'
          ,'218','244','246','253','256','273','276','297']
    
    """
        user_chosen=[]
    for item in position:
        user_chosen.append(users[item])
    """
        
    length=len(new_data)
    for i in range(length):
        if(new_data[i][0]==users[user]):#这里的user指的是在54个里面的排序
            select_data.append(new_data[i][3:])#[5:]
            pv_size=float(new_data[i][1])
    GC=[]
    CL=[]
    GG=[]
    Length_select_data = len(select_data)
    for i in range(Length_select_data):
        if(select_data[i][0]=='GC'):
            GC.append(select_data[i][2:])
        elif(select_data[i][0]=='CL'):
            CL.append(select_data[i][2:])
        else:
            GG.append(select_data[i][2:])
            
    Total_Load=[]
    temp=[]
    a=0.0
    for i in range(Length_select_data):
        if(select_data[i][0]=='GC' and select_data[i+1][0]=='CL'):
            for j in range(len(select_data[i][2:])):
                a=pd.to_numeric(select_data[i][j+2])+pd.to_numeric(select_data[i+1][j+2])
                temp.append(a)
            Total_Load.append(temp)
        elif(select_data[i][0]=='GC' and select_data[i+1][0]!='CL'):
            Total_Load.append(select_data[i][2:])
        temp=[]
        
    GC_temp=np.array(GC)
    GC_array=[]
    GC_array = GC_temp.astype(np.float)*2

    GG_temp=np.array(GG)
    GG_array=[]
    GG_array = GG_temp.astype(np.float)*2#使单位变成了千瓦
    
    Total_Load_temp=np.array(Total_Load)
    Total_Load_array=[]
    Total_Load_array = Total_Load_temp.astype(np.float32)*2#一个小时的功率KW
    
    return GG_array,Total_Load_array,pv_size

In [4]:
##滚动法求24小时的最佳SOC，选取最前一段作为SOC，这里默认充放电功率界限和效率一样
def milp_one2(T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar):
    expression=0
    tfit=[]
    model = Model()####对变量命名并且给出上下界
    var_list = np.arange(0,48,1)
    X1 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X1')#代表pg_import(pg+)，功率
    X2 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X2')#代表pg_export(pg-)
    Y1 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y1')#代表充电功率（pb+）
    Y2 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y2')#代表放电功率（pb-）
    dg=model.binary_var_list(var_list,name='dg')#决策变量电网电力流向（0：用户到电网）
    sb=model.binary_var_list(var_list,name='sb')#决策变量电池充电状态（0：放电）
    eb=model.continuous_var_list(var_list,lb=0,name='eb')#电池容量SOC
    tou=[]
    tou.extend(T_tou[i:])
    tou.extend(T_tou[0:i])#输入分时电价
    
    for j in range(48):
        #tfit.append(T_fit)
        expression+=tou[j]*X1[j]*dh#X1为pg+,X2为pg-,Y1为pb+,Y2为pb-
        expression-=T_fit*X2[j]*dh
    #约束条件（2）
        if(j+i<48):#i表示第几个时刻
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d][j+i]-efficiency_i*GG_array[d][j+i])
        else:
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d+1][j+i-48]-efficiency_i*GG_array[d+1][j+i-48])
        
    #约束条件（3）
        bat_state.append(bat_state[-1]+dh*efficiency_c*Y1[j]-(dh/efficiency_c)*Y2[j])#约束条件（3）
        eb[j]=bat_state[-1]
        model.add_constraint(eb[j]<=bat_max)#约束条件（12）
        model.add_constraint(eb[j]>=0)#约束条件（12）
    #约束条件（4）
        model.add_constraint(X1[j]-pg_bar*dg[j]<=0)
    #约束条件（5）
        model.add_constraint(X2[j]+pg_bar*dg[j]<=pg_bar)
    #约束条件（6）
        model.add_constraint(Y1[j]-pb_bar*sb[j]<=0)
    #约束条件（7）
        model.add_constraint(Y2[j]+pb_bar*sb[j]<=pb_bar)
    model.minimize(expression)#目标函数    
    sol = model.solve() #输出解
    temp = sol.get_all_values()
    return temp,bat_state[0]

In [5]:
##对用户进行364天的SOC计算
def MILP_365(Total_Load_array,GG_array,user):
    L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
 15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 10,
 30: 6,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
 44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}
    #注意这里29和30是作者剔除的，这里考虑进来了
    days=364
    efficiency_i=0.9
    efficiency_c=0.95
    dh=0.5
    pg_bar=20
    T_flat=0.235018
    T_fit=0.09
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    if(L[user]==3 or L[user]==4):
        init_state=6.5
        pb_bar=4.2
    elif(L[user]==5 or L[user]==6):
        init_state=9.8
        pb_bar=5
    elif(L[user] in [7,8,9,10]):
        init_state=14
        pb_bar=5
        
    bat_max=init_state
    bat_state=[0.5*init_state]#要输出的电池状态，这是0时刻的初始状态
    pg_import=[[] for _ in range(days)]
    pg_export=[[] for _ in range(days)]
    pb_c=[[] for _ in range(days)]
    pb_d=[[] for _ in range(days)]
    soc=[[] for _ in range(days)]
    soc[0]=[0.5*bat_max]
    
    for d in np.arange(0,days,1): 
        for i in range(48):
        #print(bat_state)
            temp=[]
            temp,before=milp_one2(T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar)
        #print(temp[96])
            if(i==47 and d<days-1):
                soc[d+1].append(abs(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144]))
            elif(i<47):     
                soc[d].append(abs(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144]))#bat_state保存的是滚动域内48个时刻都要满足电池约束，所以一个遍历完后得
        #重置，而SOC是要输出的值
            bat_state=[]
            bat_state.append(soc[d][-1])
        #d=bat_state[-1]+0.455*a[0]-(0.5/0.91)*a[0]
        #print(bat_state[-1])
            pg_import[d].append(abs(temp[0]))
            pg_export[d].append(abs(temp[48]))
            pb_c[d].append(abs(temp[96]))
            pb_d[d].append(abs(temp[144]))
    return soc,pg_import,pg_export,pb_c,pb_d

In [6]:
#将求出的一年的SOC值保存为excel格式，方便以后调用，节约时间
def save(data,path):
    wb = Workbook()
    ws = wb.active # 激活 worksheet
    [h, l] = np.array(data).shape  # h为行数，l为列数
    for i in range(h):
        row = []
        for j in range(l):
            row.append(format(data[i][j],'.4f'))
        ws.append(row)
    wb.save(path)

In [10]:
GG_array,Total_Load_array,pv_size=Read_yearRNN(2010,13)#之后RNN需后48时刻的SOC以及训练的LABEL都要是以单位为千瓦时的
soc,pg_import,pg_export,pb_c,pb_d=MILP_365(Total_Load_array,GG_array,13)
path="C:/Users/chenxihui/Desktop/code/project/processed_data/2010/socrnn-13-3.xlsx"
save(soc,path)

In [7]:
#读取之前保存的SOC值（excel形式的）
def read_data(road):
    workbook=openpyxl.load_workbook("C:/Users/chenxihui/Desktop/code/project/processed_data/2010/"+road+".xlsx")
    shenames=workbook.sheetnames
    worksheet=workbook.worksheets[0]
    name=worksheet.title 
    rows=worksheet.max_row
    columns=worksheet.max_column
    data_read=[[] for i in range(rows)]
    i=0
    for row in worksheet.rows:
        for cell in row:
            data_read[i].append(cell.value)
        i=i+1
    data_temp=np.array(data_read)
    data_array=[]
    data_array=data_temp.astype(np.float)
    return data_array

In [8]:
#为了训练RNN做的336维的数据集，大的数据集，包含了训练集和测试集
def samples(T_tou,soc_y,Total_Load_array,GG_array):
    #此版本为362天的样本数据点集合,从第一天开始算，但第一天里都是空集合，从第二天开始有数值
    T_tou_temp=[]
    for i in range(3):
        T_tou_temp.extend(T_tou)
    days=365    
    data_365=[[[] for _ in range(48)] for _ in range(days)]
    for d in range(days):
        for i in range(48):
            data_temp=[]
            GG_temp=[]
            soc_temp=[]#前48个SOC（包含当前时刻）
            ToU_temp=[]
            load_temp=[]
            if(d<363 and d>0):
                if(i==47):
                    soc_temp=soc_y[d]#soc_y是MILP的结果，从第一天开始的
                    ToU_temp=np.hstack((T_tou,T_tou))
                    load_temp=np.hstack((Total_Load_array[d],Total_Load_array[d+1]))
                    GG_temp=np.hstack((GG_array[d],GG_array[d+1]))
                else:
                    soc_temp=np.hstack((soc_y[d-1][i+1:],soc_y[d][:i+1]))
                    ToU_temp=np.hstack((T_tou[i+1:],T_tou,T_tou[:i+1]))
                    GG_temp=np.hstack((GG_array[d-1][i+1:],GG_array[d],GG_array[d+1][:i+1]))#难道d+1是
                    load_temp=np.hstack((Total_Load_array[d-1][i+1:],Total_Load_array[d],Total_Load_array[d+1][:i+1]))
            data_temp=soc_temp
            data_temp=np.hstack((data_temp,ToU_temp))
            data_temp=np.hstack((data_temp,list(map(float, GG_temp))))
            data_temp=np.hstack((data_temp,load_temp))
            data_365[d][i]=data_temp
    return data_365

In [54]:
#生成训练RNN的训练集前300天
def generate_train(year,user):
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    
    GG_array,Total_Load_array,pv_size=Read_year(year,user)
    soc_y=read_data("soc-"+str(user)+"-4")#单位为千瓦时
    soc_y=[i*2 for i in soc_y]#转换为千瓦
    data_365=samples(T_tou,soc_y,Total_Load_array,GG_array)
    a=np.array(data_365[1:302]).transpose((1,0,2))
    soc_label=[[]for i in range(360)]
    for i in range(360):
        for j in np.arange(1,48):
            soc_label[i].append(soc_y[i+1][j])#因为label是下一时刻的SOC，所以从1开始
        soc_label[i].append(soc_y[i+2][0])
    return a,soc_label

In [55]:
### 此为300天-340天数据作为同一用户的测试集
def generate_test(year,user):
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    GG_array,Total_Load_array,pv_size=Read_year(year,user)
    soc_y=read_data("soc-"+str(user)+"-4")
    soc_y=[i*2 for i in soc_y]#转换为千瓦
    data_365=samples(T_tou,soc_y,Total_Load_array,GG_array)
    
    test=np.array(data_365[1:361]).transpose((1,0,2))#测试集从第302天开始
    
    soc_label=[[]for i in range(360)]
    for i in range(360):
        for j in np.arange(1,48):
            soc_label[i].append(soc_y[i+1][j])
        soc_label[i].append(soc_y[i+2][0])
    return test,soc_label

In [25]:
### RNN的测试，对自己本身
def RNN_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func):    
    print("Testing...load model...")
    filepath = "./model/rnn"+str(year)+"-"+str(user)+".model"
    checkpoint = torch.load(filepath)
    rnn.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    rnn.eval()
    
    test,soc_label=generate_test(year,user)
    testloss=0
    with torch.no_grad():
        for step in range(40):
            test_x=torch.from_numpy(np.array(test[:,step,np.newaxis])).float().to(device)
            test_y = torch.from_numpy(np.array(soc_label[step])[:,np.newaxis,np.newaxis]).float().to(device)
            predict, h_state = rnn(test_x,h_state)
            loss_test = loss_func(predict, test_y)
            testloss+=loss_test
    print(str(user)+"testloss："+str(testloss))

In [28]:
#RNN网络的训练
def RNN_train(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func):
    rnn.train()
    a,soc_label = generate_train(year,user)
    train_loss=[]
    lr_list = []
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,100,150,200,250,300], gamma=0.6)
    max_loss,min_loss=0,20
    for step in range(N_EPOCHS):
        error=0.0
        for batch_num in range(batch_nums):#此处分成了301个batch 
            x=torch.from_numpy(np.array(a[:,batch_num,np.newaxis])).float().to(device)
            #print(x.size())48*1*336
            y=torch.from_numpy(np.array(soc_label[batch_num])[:,np.newaxis,np.newaxis]).float().to(device)
            #print(y.size())48*1*1
            x=x.cuda()
            y=y.cuda()
            optimizer.zero_grad()#清零，模型的参数梯度设成0
            prediction, h_state = rnn(x, h_state)  # RNN输出（预测结果，隐藏状态）
            #print(prediction.size())48*1*1
            h_state = h_state.detach()  # 这一行很重要，将每一次输出的中间状态传递下去(不带梯度)，将h_state分离出来，同时不影响后向计算        
            
            loss,temp_loss=0,0
            for i in range(48):
                temp_loss=abs(prediction[i][0][0]-y[i][0][0])
                if(temp_loss>max_loss):#得到预测误差最大
                    max_loss=temp_loss
                if(temp_loss<min_loss):#得到误差最小
                    min_loss=temp_loss
                loss+=temp_loss
            loss=loss/48
            
            #loss = loss_func(prediction, y)
            error += loss
            optimizer.zero_grad()#清零，模型的参数梯度设成0
            loss.backward()
            optimizer.step()
            scheduler.step()  # 学习率更新
            lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
            #torch.save(rnn.state_dict(), 'C:/Users/chenxihui/Desktop/code/project/rnn.pt')
        error /= batch_nums
        train_loss.append(error)
    print(str(user)+"  train Loss = " + str(error))#输出最后一个epoch输出的loss
    print(str(user)+"  train max Loss = " + str(max_loss))
    print(str(user)+"  train min Loss = " + str(min_loss))
    
    if not os.path.exists("./model/"):
        os.makedirs("./model/")
    filepath = "./model/rnnkw"+str(year)+"-"+str(user)+".model"
    torch.save({"model_state_dict" : rnn.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict()},
            filepath)   
    #print("Model has been saved.")
    #return train_loss[-1]

In [45]:
###step 4：对每个用户训练出的模型进行测试（在其他用户身上）
def model_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func,position):

    print("Testing...load model...")
    filepath = "./model/rnnkw"+str(year)+"-"+str(user)+".model"
    checkpoint = torch.load(filepath)
    rnn.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    rnn.eval()
    dayloss=[]
    testloss=0
    max_loss,min_loss=0,20
    for num in position:
        test,soc_label=generate_test(2010,num)
        with torch.no_grad():
            for step in range(360):
                loss_test=0
                test_x=torch.from_numpy(np.array(test[:,step,np.newaxis])).float().to(device)
                test_y = torch.from_numpy(np.array(soc_label[step])[:,np.newaxis,np.newaxis]).float().to(device)
                predict, h_state = rnn(test_x,h_state)
                loss,temp_loss=0,0
                for i in range(48):
                    temp_loss=abs(predict[i][0][0]-test_y[i][0][0])
                    if(temp_loss>max_loss):#得到预测误差最大
                        max_loss=temp_loss
                    if(temp_loss<min_loss):#得到误差最小
                        min_loss=temp_loss
                    loss+=temp_loss
                dayloss.append(loss)
                loss=loss/48
                #loss_test = loss_func(predict, test_y)#MAE 48个时刻的误差绝对值的平均值，个数是按照【】的个数，不是按照元素的个数
 #               p=np.array(predict.tolist())
 #               y=np.array(test_y.tolist())
 #               for i in range(len(p)):
 #                   loss_test+=abs(p[i]-y[i])
                testloss+=loss
    testloss=testloss/(len(position)*360)
    print("Test has been done! The LOSS is: "+str(testloss))
    print(str(user)+"  test max Loss = " + str(max_loss))
    print(str(user)+"  test min Loss = " + str(min_loss))
    print(str(user)+"  testday min Loss = " + str(min(dayloss)))
    print(str(user)+"  testday max Loss = " + str(max(dayloss)))
    return testloss

In [57]:
position=[12]#特征模型用户为82
get_indicator(position)

12  train Loss = tensor(0.7574, device='cuda:0', grad_fn=<DivBackward0>)
12  train max Loss = tensor(28.0553, device='cuda:0', grad_fn=<AbsBackward>)
12  train min Loss = tensor(0., device='cuda:0', grad_fn=<AbsBackward>)


In [59]:
position=[12]#特征模型用户为82
get_indicator(position) 

Testing...load model...
Test has been done! The LOSS is: tensor(0.9745, device='cuda:0')
12  test max Loss = tensor(11.8118, device='cuda:0')
12  test min Loss = tensor(0.0003, device='cuda:0')
12  testday min Loss = tensor(9.6042, device='cuda:0')
12  testday max Loss = tensor(87.2502, device='cuda:0')


In [58]:
#从类中返回该类的特征模型，position为一类
def get_indicator(position):
    ###超参数设置
    year=2010
    TIME_STEP = 48  # RNN时间步长
    INPUT_SIZE = 336  # RNN输入尺寸
    INIT_LR = 0.01  # 初始学习率
    N_EPOCHS = 300  # 训练回数
    Batch_size=1
    batch_nums=301
    train_loss,test_loss=[],[]
    ### RNN建立
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class RNN(nn.Module):
        def __init__(self):
            super(RNN, self).__init__()
            self.rnn = nn.RNN(
                input_size=INPUT_SIZE,
                hidden_size=20,  # RNN隐藏神经元个数20
                num_layers=1,  # RNN隐藏层个数
            )
            self.out = nn.Linear(20, 1)#全连接层

        def forward(self, x, h):
            out, h = self.rnn(x, h)
            prediction = self.out(out)
            return prediction, h
    
    ###step 1：得到分类结果，给出属于同一类的用户标号
    #position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    scores=[]
    ###step 2：处理该类中MILP、SOC，为了节约时间，将他们先处理保存，之后读取即可
    #for user in position:
        #GG_array,Total_Load_array,pv_size=Read_year(year,user)
        #soc,pg_import,pg_export,pb_c,pb_d=MILP_365(Total_Load_array,GG_array,user)
    ###step 3：对每个用户训练一个模型
    for user in position:
        rnn = RNN().to(device)
        optimizer = torch.optim.Adam(rnn.parameters(), lr=INIT_LR)
        loss_func = nn.L1Loss()
        h_state = None #之前没有这个导致前一个用户的训练影响后一个用户了
        #RNN_train(2010,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
        #RNN_test(2010,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
    ###step 4：对每个用户训练出的模型进行测试（在其他用户身上）
      
        testloss=model_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func,position)
        #print(str(user)+" : "+str(testloss))
        #scores.append(testloss)
    ###step 5:选出类的特征模型
 #  minscore=2000
 #  for i in range(len(scores)):
 #      if(minscore>scores[i]):
 #          minscore=scores[i]
 #  for i in range(len(scores)):
 #      if(minscore==scores[i]):
 #          return position[i]#得到该类中的特征模型的用户编号


In [18]:
#为节省时间，运行一遍后将数据保存直接用
final_center,position1,position2,position3,position4,position5=cluster_result(2010)

KeyboardInterrupt: 

In [15]:
def SCM(load,GG,pb_bar,bat_max):#这里的输入单位也是功率值KW，所以在算用电成本时需要乘上dh，即0.5
    bat_min,eff_i,eff_c,dh,pg_bar=0,0.9,0.95,0.5,np.max(load)
    pg_import,pg_export,pb_c,pb_d,pb_res,soc,e_b,eb_res,p_res=[],[],[],[],[],[],[0.5*bat_max],[],[]
    for d in np.arange(0,14,1):#天数设置
        for i in range(48):
            p_res.append(GG[d][i]*eff_i-load[d][i])
            #print(p_res[-1])
            #print(i)
            if(p_res[-1]>0.0):#发电量能满足需求的情况
                eb_res.append(bat_max-e_b[-1])
                #print(eb_res[-1])
                if(eb_res[-1]>0.0):#电池还能充电
                    #pb_c设置为pb_bar、p_res、将电池充满需要的电量中的最小值
                    pb_c.append(min(pb_bar,min(abs(p_res[-1])/(eff_i*eff_c),(eb_res[-1])/(dh*eff_c))))#此处改动原文eb_res[-1]/(dh*eff_c)
                    pg_export.append(abs(p_res[-1])-pb_c[-1]*eff_i*eff_c)#发电量满足充电后，卖电与eff_i有关
                    pb_d.append(0)#不放电
                    pg_import.append(0)#不用买电
                else:#电池满了，不能充电的情况
                    pg_export.append(abs(p_res[-1]))#直接卖电
                    pg_import.append(0)#不买电
                    pb_d.append(0)#不充电
                    pb_c.append(0)#不放电
            elif(p_res[-1]<0.0):#发电量不能满足需求的情况
                eb_res.append(e_b[-1]-bat_min)#电池可以放的电量
                #print(eb_res[-1])
                if(eb_res[-1]<0.01):#电池没电的情况
                    pg_import.append(abs(p_res[-1]))#买电，满足需求
                    pg_export.append(0)
                    pb_d.append(0)
                    pb_c.append(0)
                else:#电池有电的情况
                    pb_d.append(min(pb_bar,min(abs(p_res[-1])*eff_c/eff_i,(eb_res[-1]*eff_c)/(dh))))#放到电池外面的电
                    pb_c.append(0)
                    pg_import.append((abs(p_res[-1])-pb_d[-1]*eff_i/eff_c))#应该是买电，论文中的算法有错误
                    pg_export.append(0)
            else:
                pg_import.append(0.0)
                pg_export.append(0.0)
                pb_c.append(0.0)
                pb_d.append(0.0)
            e_b.append(e_b[-1]+dh*(eff_c*pb_c[-1]-pb_d[-1]/eff_c))
    return pg_import,pg_export,pb_c,pb_d,e_b

In [16]:
### RNN的测试，对自己本身
year=2010
TIME_STEP = 48  # RNN时间步长
INPUT_SIZE = 336  # RNN输入尺寸
INIT_LR = 0.01  # 初始学习率
N_EPOCHS = 300  # 训练回数
Batch_size=1
batch_nums=301
train_loss,test_loss=[],[]
    ### RNN建立
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=INPUT_SIZE,
            hidden_size=20,  # RNN隐藏神经元个数20
            num_layers=1,  # RNN隐藏层个数
            )
        self.out = nn.Linear(20, 1)#全连接层

    def forward(self, x, h):
        out, h = self.rnn(x, h)
        prediction = self.out(out)
        return prediction, h
    
rnn = RNN().to(device)
optimizer = torch.optim.Adam(rnn.parameters(), lr=INIT_LR)
loss_func = nn.L1Loss()
h_state = None 

def online(year,cluster_indicator,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func,point):    
    #print("Starting...")
    #print(point)
    filepath = "./model/rnn"+str(year)+"-"+str(cluster_indicator)+".model"#这里读取的是类特征模型
    checkpoint = torch.load(filepath)
    rnn.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    rnn.eval()   #进入测试模式，权重不变
    #生成输入
    with torch.no_grad():
        #print(np.array(point[np.newaxis,np.newaxis,:]).shape)
        x=torch.from_numpy(np.array(point[np.newaxis,np.newaxis,:])).float().to(device)
        predict, h_state = rnn(x,h_state)
    return predict    

In [31]:
import torch
import torch.nn as nn
sample = torch.tensor([[[1.,2],[1.,2]],[[1.,2],[1.,2]]])
target = torch.tensor([[[0.,0],[1.,0]],[[2.,0],[3.,0]]])
criterion = nn.L1Loss()
loss = criterion(sample, target)
loss

tensor(1.5000)

In [63]:
year=2010
#position3=[5, 8, 13, 24, 25, 27, 29, 30, 44, 50, 52]#[4, 5, 8, 13, 25, 27, 29, 30, 44, 50, 52]
#position4=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
#position2=[17, 51]
#position5=[2, 7, 20, 36, 39, 40, 48]
#position1=[0, 4, 9, 10, 11, 21, 33, 35, 37, 49, 53]
position1=[12]
cluster_indicator=12
cost1=[]
#theta=[0.025,0.05,0.1]
theta=[0]
T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
for user in position1:
    #GG_array,Total_Load_array,pv_size=Read_yearRNN(year,user)
    GG_array,Total_Load_array,pv_sizeMILP=Read_year(year,user)
    for error in theta: 
        soc_milp,pg_import_milp,pg_export_milp,pb_c_milp,pb_d_milp=MILP2(Total_LoadMILP,GG_arrayMILP,user)
        #soc_milp3,pg_import_milp3,pg_export_milp3,pb_c_milp3,pb_d_milp3=MILP2ip(Total_Load_array,GG_array,user,i)
        #pg_import,pg_export=PFA(year,user,cluster_indicator,GG_array,Total_Load_array,error)
        T_tou1=[]#PFA PP
        pg_import2,pg_export2,pb_c2,pb_d2=[],[],[],[]
        for i in np.arange(14,360,1):
            T_tou1.extend(T_tou)
            pg_import2.extend(pg_import_milp[i])
            pg_export2.extend(pg_export_milp[i])

        cost,dh,T_fit=0,0.5,0.09
        for i in range(len(pg_import2)):
            cost+=float(np.array(pg_import2[i]))*T_tou1[i]*dh-float(np.array(pg_export2[i]))*T_fit*dh
        print(str(user)+"  error:"+str(error)+"  cost: "+str(cost))
        #cost1.append(cost)

12  error:0  cost: 402.3556608808183


In [35]:
year=2010
#position3=[5, 8, 13, 24, 25, 27, 29, 30, 44, 50, 52]#[4, 5, 8, 13, 25, 27, 29, 30, 44, 50, 52]
#position4=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
#position2=[17, 51]
#position5=[2, 7, 20, 36, 39, 40, 48]
#position1=[0, 4, 9, 10, 11, 21, 33, 35, 37, 49, 53]
position1=[13]
cluster_indicator=13
cost1=[]
theta=[0]
T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
for user in position1:
    #GG_array,Total_Load_array,pv_size=Read_yearRNN(year,user)
    #GG_arrayMILP,Total_LoadMILP,pv_sizeMILP=Read_yearMILP(year,user)
    for error in theta: 
        #soc_milp,pg_import_milp,pg_export_milp,pb_c_milp,pb_d_milp=MILP2(Total_LoadMILP,GG_arrayMILP,user)
        #soc_milp3,pg_import_milp3,pg_export_milp3,pb_c_milp3,pb_d_milp3=MILP2ip(Total_Load_array,GG_array,user,i)
        #pg_import,pg_export=PFA(year,user,cluster_indicator,GG_array,Total_Load_array,error)
        T_tou1=[]#PFA PP
        #pg_import2,pg_export2,pb_c2,pb_d2=[],[],[],[]
        for i in np.arange(14,360,1):
            T_tou1.extend(T_tou)
            pg_import2.extend(pg_import[i])
            pg_export2.extend(pg_export[i])

        cost,dh,T_fit=0,0.5,0.09
        for i in range(len(pg_import2)):
            cost+=float(np.array(pg_import2[i]))*T_tou1[i]*dh-float(np.array(pg_export2[i]))*T_fit*dh
        print(str(user)+"  error:"+str(error)+"  cost: "+str(cost))
        #cost1.append(cost)

13  error:0  cost: 2167.349359390805


In [61]:
#分完类后对用户执行线上执行策略，执行前得做输入数据集
    #cluster_indicator=indicators[newuser_cluster]
def PFA(year,user,cluster_indicator,GG_array,Total_Load_array,theta):
    #cluster_indicator=indicators[newuser_cluster]
    #user=cluster_indicator  # 13，"82"
    #GG_array,Total_Load_array,pv_size=Read_year(year,user)#USER[13]=82
    #soc_milp,pg_import_milp,pg_export_milp,pb_c_milp,pb_d_milp=MILP2(Total_Load_array,GG_array,user)
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
           0.38588,0.38588, 0.38588,0.38588, 
          0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
          0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
          0.37147,0.37147,0.37147,0.37147,
          0.2134,0.2134,0.2134,0.2134]
    days=365
    eff_i,eff_c,dh,bat_min,T_flat,T_fit=0.9,0.95,0.5,0,0.235,0.09
    points365=[[[] for _ in range(48)] for _ in range(days)]
    pg_import=[[0 for i in range(48)] for _ in range(days)]
    pg_export=[[0 for i in range(48)] for _ in range(days)]
    pb_c=[[0 for i in range(48)] for _ in range(days)]
    pb_d=[[0 for i in range(48)] for _ in range(days)]
    L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
     15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 10,
     30: 6,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
     44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}
    pg_bar=20
    if(L[user]==3 or L[user]==4):
        init_state=6.5
        pb_bar=4.2
    elif(L[user]==5 or L[user]==6):
        init_state=9.8
        pb_bar=5
    elif(L[user] in [7,8,9,10]):
        init_state=14
        pb_bar=5
    bat_max=init_state
    bat_min=0
    
    pgi_scm,pgo_scm,pb_cscm,pb_dscm,e_bscm=SCM(Total_Load_array,GG_array,pb_bar,bat_max)
    e_bscm=[i*2 for i in e_bscm]#将单位转换为千瓦
    #soc=read_data("soc-"+str(user)+"-3")
    soc,pg_import,pg_export,pb_c,pb_d=MILP2(Total_Load_array,GG_array,user)#这是从第14天开始算的，是一维的数组
    soc=[i*2 for i in soc]
    pred=[e_bscm[671]]#0，第14天最后一个时段
    soc_init=e_bscm[623:671]
    loss=0
    losspoint,lossday=[],[]
    #soc_y=read_data("soc-"+str(13)+"-3")
    for d in np.arange(14,360,1):#346
        loss_temp=0
        for i in range(48):#！！！！！要做出决策的时段
    #从用户第三个星期开始运行
    #制作线上执行模型的第一个点336维,第15天的0:30
            data_temp,GG_temp,soc_temp,ToU_temp,load_temp=[],[],[],[],[]#前48个SOC（包含当前时刻）
            for j in range(47):
                soc_init[j]=soc_init[j+1]#e_bscm[624:671]
            if(d==14 and i==0):
                soc_init[47]=pred[-1]
            else:
                soc_init[47]=pred[-1]*2#SOC更新
                
            if(i==0):
                ToU_temp=np.hstack((T_tou,T_tou))
                GG_temp=np.hstack((GG_array[d-1],[j+np.random.uniform(-theta*j, theta*j) for j in GG_array[d]]))
                load_temp=np.hstack((Total_Load_array[d-1],[j+np.random.uniform(-theta*j, theta*j) for j in Total_Load_array[d]]))
                #load_temp=np.hstack((Total_Load_array[d-1],Total_Load_array[d-7]))
            else:
                ToU_temp=np.hstack((T_tou[i:],T_tou,T_tou[:i]))
                GG_temp=np.hstack((GG_array[d-1][i:],GG_array[d],GG_array[d+1][:i]))#难道d+1是
                for j in np.arange(48,96,1):
                    GG_temp[j]=GG_temp[j]+np.random.uniform(-theta*GG_temp[j], theta*GG_temp[j])#*GG_temp[j]
                load_temp=np.hstack((Total_Load_array[d-1][i:],Total_Load_array[d][:i],
                                     [j+np.random.uniform(-theta*j, theta*j) for j in Total_Load_array[d][i:]],
                                     [j+np.random.uniform(-theta*j, theta*j) for j in Total_Load_array[d+1][:i]]))
                #load_temp=np.hstack((Total_Load_array[d-1][i:],Total_Load_array[d][:i],
                                     #Total_Load_array[d-7][i:],Total_Load_array[d+1-7][:i]))
            data_temp=soc_init
            data_temp=np.hstack((data_temp,ToU_temp))
            data_temp=np.hstack((data_temp,list(map(float, GG_temp))))
            data_temp=np.hstack((data_temp,load_temp))
            points365[d][i]=data_temp#线上执行策略时数据的输入
    #返回线上执行预测的
            predict=online(year,cluster_indicator,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func,points365[d][i]).item()
            soc_before=pred[-1]
            if(predict>bat_max or abs(predict-soc_before)>pb_bar):#预测结果有风险的情况
                pg_import,pg_export,pb_c,pb_d,pred=SCM2(Total_Load_array,GG_array,pg_import,pg_export,pb_c,pb_d,pb_bar,bat_max,bat_min,eff_i,eff_c,dh,pg_bar,pred,d,i,theta)
            else:#预测结果没问题的情况
                pg_import,pg_export,pb_c,pb_d=update(predict,GG_temp,load_temp,user,GG_array,Total_Load_array,d,i,pred
                  ,pg_import,pg_export,pb_c,pb_d,bat_max,eff_i,eff_c,dh,theta)
                pred.append(predict)
            loss_temp+=abs(pred[-1]-soc[(d-14)*48+i+1])
            if(i==47):
                lossday.append(loss_temp)
            loss+=abs(pred[-1]-soc[(d-14)*48+i+1])
            losspoint.append(abs((pred[-1]-soc[(d-14)*48+i+1])))
    print(str(user)+"  PFA total Loss = " + str(loss))
    print(str(user)+"  test max Loss = " + str(max(losspoint)))
    print(str(user)+"  test min Loss = " + str(min(losspoint)))
    print(str(user)+"  testday min Loss = " + str(min(lossday)))
    print(str(user)+"  testday max Loss = " + str(max(lossday)))
    return pg_import,pg_export

In [37]:
soc_y=read_data("soc-"+str(13)+"-3")
soc_y[0][0]

Exception ignored in: <function OutputStream.__del__ at 0x000002AE9CA94488>
Traceback (most recent call last):
  File "D:\Users\chenxihui\Anaconda3\envs\pytorch\lib\site-packages\cplex\_internal\_ostream.py", line 96, in __del__
KeyboardInterrupt


7.0

In [18]:
##对用户进行364天的SOC计算
def MILP2(Total_Load_array,GG_array,user):
    L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
 15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 10,
 30: 6,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
 44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}
    #注意这里29和30是作者剔除的，这里考虑进来了
    
    days=364
    efficiency_i=0.9
    efficiency_c=0.95
    dh=0.5
    pg_bar=20
    T_flat=0.235018
    T_fit=0.09
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    if(L[user]==3 or L[user]==4):
        init_state=6.5
        pb_bar=4.2
    elif(L[user]==5 or L[user]==6):
        init_state=9.8
        pb_bar=5
    elif(L[user] in [7,8,9,10]):
        init_state=14
        pb_bar=5
        
    bat_max=init_state
    #bat_state=[0.5*init_state]#要输出的电池状态，这是0时刻的初始状态
    bat_state=[0]
    pg_import=[[0 for i in range(48)] for _ in range(days)]
    pg_export=[[0 for i in range(48)] for _ in range(days)]
    pb_c=[[0 for i in range(48)] for _ in range(days)]
    pb_d=[[0 for i in range(48)] for _ in range(days)]
    soc=[]#电池状态用一维数组接收
    #soc[0]=[0.5*bat_max]
    soc.append(0)#0是初试的
    
    for d in np.arange(14,360,1): 
        for i in range(48):
        #print(bat_state)
            temp=[]
            temp,before=milp_one2(T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar)
        #print(temp[96])
            soc.append(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144])#bat_state保存的是滚动域内48个时刻都要满足电池约束，所以一个遍历完后得
        #重置，而SOC是要输出的值
            bat_state=[]
            bat_state.append(soc[-1])
        #d=bat_state[-1]+0.455*a[0]-(0.5/0.91)*a[0]
        #print(bat_state[-1])
            pg_import[d][i]=(abs(temp[0]))
            pg_export[d][i]=(abs(temp[48]))
            pb_c[d][i]=(abs(temp[96]))
            pb_d[d][i]=(abs(temp[144]))
    return soc,pg_import,pg_export,pb_c,pb_d

def SCM2(load,GG,pg_import,pg_export,pb_c,pb_d,pb_bar,bat_max,bat_min,eff_i,eff_c,dh,pg_bar,pred,d,i,theta):
    p_res=GG[d][i]*eff_i-load[d][i]
    if(p_res>0.0):#发电量能满足需求的情况
        eb_res=bat_max-pred[-1]
        if(eb_res>0.0):#电池还能充电
            #pb_c设置为pb_bar、p_res、将电池充满需要的电量中的最小值
            pb_c[d][i]=min(pb_bar,min(abs(p_res)*eff_i,(eb_res)/(dh*eff_c)))#此处改动原文eb_res[-1]/(dh*eff_c)
            pg_export[d][i]=abs(p_res)-pb_c[d][i]/eff_i#发电量满足充电后，卖电与eff_i有关
            pb_d[d][i]=(0)#不放电
            pg_import[d][i]=(0)#不用买电
        else:#电池满了，不能充电的情况
            pg_export[d][i]=(abs(p_res))#直接卖电
            pg_import[d][i]=(0)#不买电
            pb_d[d][i]=(0)#不充电
            pb_c[d][i]=(0)#不放电
    elif(p_res<0.0):#发电量不能满足需求的情况
        eb_res=(pred[-1]-bat_min)#电池可以放的电量
        if(eb_res<0.01):#电池没电的情况
            pg_import[d][i]=(abs(p_res))#买电，满足需求
            pg_export[d][i]=(0)
            pb_d[d][i]=(0)
            pb_c[d][i]=(0)
        else:#电池有电的情况
            pb_d[d][i]=(min(pb_bar,min(abs(p_res)/eff_i,(eb_res*eff_c)/(dh))))#放到电池外面的电
            pb_c[d][i]=(0)
            pg_import[d][i]=((abs(p_res)-pb_d[d][i]*eff_i))#应该是买电，论文中的算法有错误
            pg_export[d][i]=(0)
    else:
        pg_import[d][i]=(0.0)
        pg_export[d][i]=(0.0)
        pb_c[d][i]=(0.0)
        pb_d[d][i]=(0.0)
    pred.append(pred[-1]+(eff_c*pb_c[d][i]-pb_d[d][i]/eff_c))#此处没有乘以dh，因为单位为千瓦
    #pred.append(pred[-1]+dh*(eff_c*pb_c[d][i]-pb_d[d][i]/eff_c))
    return pg_import,pg_export,pb_c,pb_d,pred

def update(predict,ggtemp,loadtemp,user,GG_array,Total_Load_array,d,i,pred,
           pg_import,pg_export,pb_c,pb_d,bat_max,eff_i,eff_c,dh,theta):
############电池单位为千瓦时，其他都是以千瓦为单位####
    soc_before=pred[-1]
    theta1=(predict-soc_before)
    if(theta1>=0.0):#要充电
        pb_c[d][i]=(theta1)/(dh*eff_c)
        pb_d[d][i]=(0)
        theta2=GG_array[d][i]*eff_i-Total_Load_array[d][i]-pb_c[d][i]*eff_c*eff_i#下一时刻实际发电量是否满足
        
        if(theta2>=0):#发电量够了,卖电
            pg_import[d][i]=0
            pg_export[d][i]=(theta2)#卖出多余的电
        else:#thata3<0，剩余发电量不能满足负载，买电
            pg_import[d][i]=(abs(theta2))
            pg_export[d][i]=(0)
    else:#要放电
        pb_c[d][i]=(0)
        pb_d[d][i]=abs(theta1*eff_c)/(dh)
        theta2=GG_array[d][i]*eff_i-Total_Load_array[d][i]-pb_d[d][i]*eff_i/eff_c#下一时刻实际发电量是否满足
        if(theta2>=0):#放的电能满足负载
            pg_export[d][i]=(theta2)
            pg_import[d][i]=[0]
        else:
            pg_import[d][i]=abs(theta2)
            pg_export[d][i]=(0)
    return pg_import,pg_export,pb_c,pb_d

In [19]:
##滚动法求24小时的最佳SOC，选取最前一段作为SOC，这里默认充放电功率界限和效率一样
def milp_oneip(theta,T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar):
    expression=0
    tfit=[]
    model = Model()####对变量命名并且给出上下界
    var_list = np.arange(0,48,1)
    X1 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X1')#代表pg_import(pg+)，功率
    X2 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X2')#代表pg_export(pg-)
    Y1 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y1')#代表充电功率（pb+）
    Y2 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y2')#代表放电功率（pb-）
    dg=model.binary_var_list(var_list,name='dg')#决策变量电网电力流向（0：用户到电网）
    sb=model.binary_var_list(var_list,name='sb')#决策变量电池充电状态（0：放电）
    eb=model.continuous_var_list(var_list,lb=0,name='eb')#电池容量SOC
    tou=[]
    tou.extend(T_tou[i:])
    tou.extend(T_tou[0:i])#输入分时电价
    
    for j in range(48):
        #tfit.append(T_fit)
        expression+=tou[j]*X1[j]*dh#X1为pg+,X2为pg-,Y1为pb+,Y2为pb-
        expression-=T_fit*X2[j]*dh
    #约束条件（2）

        if(j+i<48):#i表示第几个时刻
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d-7][j+i]-efficiency_i*(GG_array[d][j+i]+np.random.uniform(-theta*GG_array[d][j+i], theta*GG_array[d][j+i])))
        else:
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d+1-7][j+i-48]-efficiency_i*(GG_array[d+1][j+i-48]+np.random.uniform(-theta*GG_array[d][j+i-48], theta*GG_array[d][j+i-48])))

    #约束条件（3）
        bat_state.append(bat_state[-1]+dh*efficiency_c*Y1[j]-(dh/efficiency_c)*Y2[j])#约束条件（3）
        eb[j]=bat_state[-1]
        model.add_constraint(eb[j]<=bat_max)#约束条件（12）
        model.add_constraint(eb[j]>=0)#约束条件（12）
    #约束条件（4）
        model.add_constraint(X1[j]-pg_bar*dg[j]<=0)
    #约束条件（5）
        model.add_constraint(X2[j]+pg_bar*dg[j]<=pg_bar)
    #约束条件（6）
        model.add_constraint(Y1[j]-pb_bar*sb[j]<=0)
    #约束条件（7）
        model.add_constraint(Y2[j]+pb_bar*sb[j]<=pb_bar)
    model.minimize(expression)#目标函数    
    sol = model.solve() #输出解
    temp = sol.get_all_values()
    return temp,bat_state[0]

In [20]:
##对用户进行364天的SOC计算
def MILP2ip(Total_Load_array,GG_array,user,theta):
    L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
 15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 10,
 30: 6,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
 44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}
    #注意这里29和30是作者剔除的，这里考虑进来了
    days=364
    efficiency_i=0.9
    efficiency_c=0.95
    dh=0.5
    pg_bar=20
    T_flat=0.235018
    T_fit=0.09
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    if(L[user]==3 or L[user]==4):
        init_state=6.5
        pb_bar=4.2
    elif(L[user]==5 or L[user]==6):
        init_state=9.8
        pb_bar=5
    elif(L[user] in [7,8,9,10]):
        init_state=14
        pb_bar=5
        
    bat_max=init_state
    #bat_state=[0.5*init_state]#要输出的电池状态，这是0时刻的初始状态
    bat_state=[0]
    pg_import=[[0 for i in range(48)] for _ in range(days)]
    pg_export=[[0 for i in range(48)] for _ in range(days)]
    pb_c=[[0 for i in range(48)] for _ in range(days)]
    pb_d=[[0 for i in range(48)] for _ in range(days)]
    soc=[]#电池状态用一维数组接收
    #soc[0]=[0.5*bat_max]
    soc.append(0)#0是初试的
    
    for d in np.arange(14,360,1): 
        for i in range(48):
        #print(bat_state)
            temp=[]
            temp,before=milp_oneip(theta,T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar)
        #print(temp[96])
            soc.append(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144])#bat_state保存的是滚动域内48个时刻都要满足电池约束，所以一个遍历完后得
        #重置，而SOC是要输出的值
            bat_state=[]
            bat_state.append(soc[-1])
        #d=bat_state[-1]+0.455*a[0]-(0.5/0.91)*a[0]
        #print(bat_state[-1])
            pg_import[d][i]=(abs(temp[0]))
            pg_export[d][i]=(abs(temp[48]))
            pb_c[d][i]=(abs(temp[96]))
            pb_d[d][i]=(abs(temp[144]))
    return soc,pg_import,pg_export,pb_c,pb_d

In [24]:
soc_milp3,pg_import_milp3,pg_export_milp3,pb_c_milp3,pb_d_milp3=MILP2ip(Total_Load_array,GG_array,user)
len(soc_milp)

NameError: name 'Total_Load_array' is not defined

In [77]:
T_tou1=[]#PFA PP
pg_import2,pg_export2,pb_c2,pb_d2=[],[],[],[]
for i in np.arange(14,360,1):
    T_tou1.extend(T_tou)
    pg_import2.extend(pg_import[i])
    pg_export2.extend(pg_export[i])

cost,dh,T_fit=0,0.5,0.09
for i in range(len(pg_import2)):
    cost+=pg_import2[i]*T_tou1[i]*dh-pg_export2[i]*T_fit*dh
cost

2912.4660509340183

In [78]:
load=[]#MILP PP
GGG=[]
T_tou1=[]
pg_import3,pg_export3,pb_c3,pb_d3,soc3=[],[],[],[],[]
for i in np.arange(14,360,1):
    load.extend(Total_Load_array[i])
    GGG.extend(GG_array[i])
    T_tou1.extend(T_tou)
    pg_import3.extend(pg_import_milp[i])
    pg_export3.extend(pg_export_milp[i])
    pb_c3.extend(pb_c_milp[i])
    pb_d3.extend(pb_d_milp[i])
cost,dh,T_fit=0,0.5,0.09
for i in range(len(pg_import3)):
    cost+=pg_import3[i]*T_tou1[i]*dh-pg_export3[i]*T_fit*dh
cost

1912.2763698200522

In [80]:
load=[]
GGG=[]
T_tou1=[]
pg_import4,pg_export4,pb_c4,pb_d4,soc4=[],[],[],[],[]
for i in np.arange(14,360,1):
    T_tou1.extend(T_tou)
    pg_import4.extend(pg_import_milp3[i])
    pg_export4.extend(pg_export_milp3[i])
cost,dh,T_fit=0,0.5,0.09
for i in range(len(pg_import4)):
    cost+=pg_import4[i]*T_tou1[i]*dh-pg_export4[i]*T_fit*dh
cost

2083.212213698412